In [ ]:
import torch.nn as nn

In [ ]:
import pandas as pd
import torch
import numpy as np
import string
newsdf=pd.read_csv('/content/corona_fake.csv')
titles=newsdf['title']
text=newsdf['text']
labels=newsdf['label']
letters=string.ascii_letters+' '
def enum1(x):
    for i in range(len(x)):
        yield (i+1,x[i])
vocab={letter:i for i,letter in enum1(letters)}
def word2tensor(word):
  tens=[vocab[i] for i in word if i in letters]
  return torch.tensor(tens)
def stack(data):
  result=[]
  for i in range(len(data)):
    try:
      result.append(word2tensor(data[i]))
    except:
      result.append(torch.zeros(1))
  return result
tensortext=stack(text)
tensorlabel=[]
for label in labels:
  if label=='Fake' or label=='fake':
    tensorlabel.append(torch.ones(1))
  elif label=='TRUE':
    tensorlabel.append(torch.zeros(1))
  else:
    print(label,'n')
    tensorlabel.append(-1)
tensortitles=stack(titles)
def greatest_len(x):
  great=len(x[0])
  for i in x:
    if len(i)>great:
      great=i
    return great
def pad_to(x,length):
  return torch.nn.functional.pad(x,(0,length-len(x)))
def clean(thing,length):
  new=[]
  for sentence in thing:
    new.append(pad_to(sentence,length))
  return new

nan n
nan n
nan n
nan n
nan n


In [ ]:
titlemax=150
textmax=400
ptext=clean(tensortext,textmax)
ptitle=clean(tensortitles,titlemax)
len(ptitle),len(ptext),len(tensorlabel)
all_data=[]
for j in range(len(ptitle)):
  if tensorlabel[j]<0:
    print(tensorlabel[j])
    pass
  elif sum(ptext).sum().data==0 and sum(ptitle).sum().data==0:
    pass 
  else:
    all_data.append((ptitle[j],ptext[j],tensorlabel[j]))


-1
-1
-1
-1
-1


In [ ]:
first=torch.tensor([])
torch.cat((first,torch.tensor([[1,2,3]]),torch.tensor([[1,2,3]])))

tensor([[1., 2., 3.],
        [1., 2., 3.]])

In [ ]:
def batcher(x,batch):
  count=batch
  batchet=[]
  batchd=[]
  for i in range(0,len(x),batch):
    tempd=[]
    tempt=[]
    for j in range(batch):
      data,target=x[i+j]
      tempt.append(target)
      tempd.append(data)
    batchet.append(tempt)
    batchd.append(tempd)
    count+=batch
  return batches
def val(data,tmodel):
  tmodel.eval()
  scores=[]
  with torch.no_grad():
    for i,j in data:
      scores.append([tmodel(i,torch.randn(1,1,100)),j])
  correct=0
  for i in scores:
    if torch.round(i[0])==i[1]:
      correct+=1
  tmodel.train()
  return correct/len(scores)

def score(scores):
  correct=0
  for i in scores:
    if torch.round(i[0])==i[1]:
      correct+=1
  return correct/len(scores)

train_data=all_data[264:]
val_data=all_data[:264]

In [ ]:
linearmodel.eval()
count=0
with torch.no_grad():
  for i1,i2,j in val_data:
    i=torch.cat((i1,i2))
    if torch.round(linearmodel(i))==j:
      count+=1
print(count/len(val_data))
linearmodel.train()

0.7159090909090909


Linearclassifier(
  (embed): EmbeddingBag(54, 10, mode=mean)
  (fc1): Linear(in_features=10, out_features=1, bias=True)
  (fc2): Linear(in_features=550, out_features=1, bias=True)
)

In [ ]:
class Linearclassifier(nn.Module):
  def __init__(self,embed_dim=10,seq=550,out=1):
    super().__init__()
    self.embed=nn.EmbeddingBag(len(vocab)+1,embed_dim)
    self.fc1=nn.Linear(embed_dim,out)
    self.fc2=nn.Linear(seq,out)
  def forward(self,x):
    xe=self.embed(x.long(),torch.LongTensor([0,550]))
    x1=torch.sigmoid(self.fc1(xe))
    #x2=torch.sigmoid(self.fc2(x1))
    return torch.mean(x1)

linearmodel=Linearclassifier()
lossflinear=torch.nn.BCELoss()
optim_linear=torch.optim.Adam(linearmodel.parameters(),lr=0.01)
def train2():
  for epoch in range(200):
    counter=pbar(1,len(train_data),True)
    for data1,data2,target in train_data:
      data=torch.cat((data1,data2))
      counter.frac()
      pred=linearmodel(data)
      loss=lossflinear(pred.reshape(-1),target)
      linearmodel.zero_grad()
      optim_linear.zero_grad()
      loss.backward()
      optim_linear.step()
    if epoch%10==0:
      linearmodel.eval()
      count=0
      with torch.no_grad():
        for i1,i2,j in val_data:
          i=torch.cat((i1,i2))
          if torch.round(linearmodel(i))==j:
            count+=1
      print(count/len(val_data))
      linearmodel.train()
      print(loss)
train2()

In [ ]:
class Classifiergru(nn.Module):
  def __init__(self,mid=100,seq=550,out=1,embed_dim=10):
    super().__init__()
    self.seq=seq
    self.embed_dim=embed_dim
    self.embed=nn.Embedding(len(vocab)+1,embed_dim)
    self.gru=nn.GRU(embed_dim,mid)
    self.fc=nn.Linear(mid,out)
  def forward(self,x,hidden):
    x=self.embed(x.long()).float().reshape(self.seq,1,-1)
    gruout,hidden=self.gru(x,hidden)
    return torch.sigmoid(self.fc(nn.functional.relu(hidden)))
model=Classifiergru()
lossf=torch.nn.BCELoss()
optimizer=torch.optim.Adam(model.parameters(),0.01)
def train1():
  for epoch in range(7):
    bar=pbar(1,len(train_data),True)
    for data1,data2,target in train_data:
      data=torch.cat((data1,data2))
      bar.frac()
      hidden=torch.randn(1,1,100)
      pred=model(data,hidden)
      try:
        loss=lossf(pred.reshape(-1),target.float())
      except:
        print(pred.reshape(-1))
        print(data)
      model.zero_grad()
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
    print(val(val_data,model))
    print(loss)
train1()

In [ ]:
class Grudiscrete(nn.Module):
  def __init__(self, embed_dim=10,hidden_s=200,seq1=400,seq2=150):
    super().__init__()
    self.gru1=nn.GRU(embed_dim,hidden_s)
    self.gru2=nn.GRU(embed_dim,hidden_s)
    self.embed=nn.Embedding(len(vocab)+1,embed_dim)
    self.fc1=nn.Linear(hidden_s,1)
    self.fc2=nn.Linear(hidden_s,1)
    self.fc3=nn.Linear(2,1)
    self.seq1=seq1
    self.seq2=seq2
    self.hidden_s=hidden_s
  def forward(self,x,x1,hidden):
    embedded1=self.embed(x.long()).reshape(self.seq1,1,-1)
    embedded2=self.embed(x1.long()).reshape(self.seq2,1,-1)
    encode1,h1=self.gru1(embedded1,hidden)
    encode2,h2=self.gru2(embedded2,hidden)
    out1=nn.functional.relu(self.fc1(h1))
    out2=nn.functional.relu(self.fc2(h2))
    return torch.sigmoid(self.fc3(torch.cat((out1,out2),dim=2)))


def train3():
  model_both=Grudiscrete()
  loss_func=nn.BCELoss()
  optimizer=torch.optim.Adam(model_both.parameters())
  for i in range(50):
    bar=pbar(1,len(train_data),True)
    for title,text,target in train_data:
      title=title.reshape(150,-1)
      text=text.reshape(400,-1)
      pred=model_both(text,title,torch.zeros(1,1,200))
      loss=loss_func(pred.reshape(-1),target)
      model_both.zero_grad()
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      bar.frac()
    model_both.eval()
    count=0
    with torch.no_grad():
      for test_title,test_text,test_target in val_data:
        title=title.reshape(150,-1)
        text=text.reshape(400,-1)
        prediction=model_both(test_text,test_title,torch.zeros(1,1,200))
        print(prediction.reshape(-1),test_target)
        if torch.round(prediction.reshape(-1))==target:
          count+=1
    model_both.train()
    print(count/len(val_data))
    print(loss)
train3()


In [ ]:
import torch
import torch.nn as nn
gru2=nn.GRU(2,3,2)
input2=torch.ones(1,1,2)
h_2=torch.ones(2,1,3)
out2,h2=gru2(input2,h_2)
print(h2.shape)
gru1=nn.GRU(2,3)
input1=torch.ones(1,1,2)
h_1=torch.ones(1,1,3)
out1,h1=gru1(input1,h_1)
print(h1.shape)

torch.Size([2, 1, 3])
torch.Size([1, 1, 3])


In [ ]:
import sys as s
import time
import math
def p(thing):
  s.stdout.write(thing)
class pbar:
  def __init__(self,length,total,frac=False):
    self.length=length
    self.total=total
    if not frac:
      p('|'+'.'*length+'|')
    else:
      pass
    self.count=0
    self.thresh=total//length
    self.all_count=0
    self.n=0
  def step(self):
    self.count+=1
    if self.all_count==self.length/self.thresh:
      p(p('|'+'-'*length+'|'))
    if self.count==self.thresh:
      self.count=0
      p('\b'*(length+2))
      p('|'+'-'*(self.length-self.all)+'.'*length+'|')
      self.all_count+=1
  def frac(self):
    if self.n==self.total-1:
      p('\n')
      return
    if self.n>0:
      p('\b'*self.len)
    string=f'{self.n}/{self.total}'
    p(string)
    self.len=len(string)
    self.n+=1
    

In [ ]:
class Stackcell(nn.Module):
  def __init__(self,in_size,mid,out,bias1,bias2):
    self.gru1=nn.GRUCell(in_size,mid,bias=bias1)
    self.gru2=nn.GRUCell(in_size,mid,bias=bias2)
    self.fc=nn.Linear(mid,out)
  def forward(self,x,hidden1,hidden2,x2):
    h1=self.gru1(x,hidden1)
    h2=self.gru2(x2,hidden2)
    return self.fc(torch.cat((h1,h2)))
    
class Stack(nn.Module):
  def __init__(self,in_size,mid,out):
    super().__init__()


SyntaxError: ignored

In [ ]:
torch.sigmoid(torch.tensor([1.],requires_grad=True))

tensor([0.7311], grad_fn=<SigmoidBackward>)

In [ ]:
a=[torch.tensor([i for i in range(10)]) for k in range(10)]

In [ ]:
embedbag=nn.EmbeddingBag(3,2,mode='sum')
embedbag(torch.ones(3,2).long(),None)

tensor([[ 0.6240, -2.3776],
        [ 0.6240, -2.3776],
        [ 0.6240, -2.3776]], grad_fn=<EmbeddingBagBackward>)

In [ ]:
zeros=torch.zeros(2,3)
ones=torch.ones(2,3)
torch.cat((zeros,ones),dim=1)

tensor([[0., 0., 0., 1., 1., 1.],
        [0., 0., 0., 1., 1., 1.]])

In [ ]:
torch.cat((torch.ones(1,1),torch.ones(1,1)),dim=1)

tensor([[1., 1.]])

In [ ]:
template=torch.randn(5)
template[0]=torch.randn(2,3)

RuntimeError: ignored

In [ ]:
a=torch.cat((torch.tensor([]),torch.randn(2,3)))
torch.cat((a,torch.ones(2,3))).reshape(2,2,3)

tensor([[[ 0.3971,  0.5770, -0.2544],
         [-2.2514,  1.1398,  0.3002]],

        [[ 1.0000,  1.0000,  1.0000],
         [ 1.0000,  1.0000,  1.0000]]])

In [ ]:
a=torch.zeros(1,3)
b=torch.ones(1,3)
torch.cat((a,b),dim=1),torch.cat((a,b),dim=1).shape

(tensor([[0., 0., 0., 1., 1., 1.]]), torch.Size([1, 6]))

In [ ]:
torch.tensor([])[1]

IndexError: ignored

In [ ]:
x=torch.randn(5,10)
torch.nn.functional.softmax(x,dim=1)[0].sum()


tensor(1.)

In [ ]:
tes=torch.ones(5,2)
tes.sum()

tensor(10.)

In [ ]:
a=torch.ones(400)
b=torch.zeros(150)
torch.cat((a,b))

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 

In [ ]:
torch.tensor([[1,2,3,4]]).sum()

tensor(10)

In [ ]:
torch.ones(550,100).sum(dim=0).shape

torch.Size([100])

In [ ]:
if torch.zeros(550,404).sum()==0:
  print('fgsyf')

fgsyf
